# University Campus Venue Analysis

## Week 1

### Problem description

This Capstone project will investigate the venues close to some university campuses, such as the MIT and Harvard campus in Cambridge, MA, the Columbia campus in Manhattan, NY, the NTNU campus in Trondheim, NO and the EPFL campus in Lausanne, CH. 

The universities are different in size and academic scope, and this should be reflected in the analysis. Although cultural differences may also influence the venues nearby campuses, this will be excluded from the analysis, under the assumption that all campuses have a diverse, international student body.

The goal of the project is to identify what universities have similar venues and cluster them in qualitatively reasonable clusters.

### Data description

The required data will be coordinates for the different campuses, which can be found manually on Google Maps, the venues nearby the cmapuses, found using the Foursquare API, the type of university, found using Wikipedia, and the size of the university (in terms of students), also found using Wikipedia. Additional universities, other than the ones mentioned above, may be included for a even more thorough analysis of university campuses.

The data will be used to extract reasonable features. The features will be encoded in such a way that the scaling problem is not an issue, e.g. using one-hot encoding for categorical variables, such as university type. The features will also be scaled to avoid improper weight on the different features. If this is not done, the size feature is likely to dominate the feature space, as it would likely have dominated being the scales of thousands, compared to the one-hot-encoded university type being either 0 or 1. After applying a clustering algorithm, for instance, kmeans, for different k. A k will be chosen based on the resulting clusters having the most reasonable qualitative description. Although a geographical visluzal

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone project course!')

Hello Capstone project course!
